In [ ]:
# pip install the required packages if running in Colab
try:
    import google.colab
    IN_COLAB = True
    %pip install summerepi2==1.0.4
    %pip install estival==0.1.7
except:
    IN_COLAB = False

In [ ]:
# Standard imports, plotting option and constant definition
from datetime import datetime
from typing import List, Union
import pandas as pd
import plotly.express as px
import numpy as np
import random

from summer2 import CompartmentalModel, Stratification
from summer2.parameters import Parameter, Time, Data

pd.options.plotting.backend = "plotly"
random.seed(10)

In [ ]:
initial_population = 32657100
start_date_int = 0
end_date_int = 500.

# Model


In [ ]:
unstratified_compartments = ["S", "I1", "I2", "R"]

In [ ]:
def build_unstratified_model() -> CompartmentalModel:
    """
    Create a compartmental model, with compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        parameters: Flow parameters
    Returns:
        A compartmental model currently without stratification applied
    """
    model = CompartmentalModel(
        times=(start_date_int, end_date_int),
        compartments=unstratified_compartments,
        infectious_compartments=["I1", "I2"],
        ref_date=datetime(2023, 1, 1)
    )

    infectious_seed = Parameter("infectious_seed")
    infectious_period = Parameter("infectious_period")

    model.set_initial_population(
        distribution=
        {
            "S": initial_population - infectious_seed, 
            "I1": infectious_seed,
        }
    )
    
    # Susceptible people can get infected
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("beta"), 
        source="S",
        dest="I1",
    )  

    # People transition through the different infection stages
    model.add_transition_flow(
        name="progression",
        fractional_rate=2. / infectious_period,
        source="I1",
        dest="I2",
    )

    # Infectious people recover after some time spent infectious
    model.add_transition_flow(
        name="recovery",
        fractional_rate=2. / infectious_period,
        source="I2",
        dest="R",
    )
    
    # Only a proportion of new cases are identified as cases
    model.request_output_for_flow(
        name="incidence",
        flow_name="infection",
    )

    return model

In [ ]:
def get_infectiousness_stratification() -> Stratification:
    """
    Create a summer stratification object that stratifies all of the infectious compartments into
    strata, which are intended to represent different levels of infectiousness.

    Returns:
        A summer stratification object to represent age stratification (not yet applied)
    """   
    # Some preparation
    strata = ["subspreader", "superspreader"]
    infectious_comps = ["I1", "I2"]

    # Create the stratification, just naming the age groups by their starting value
    strat = Stratification(
        name="infectiousness", 
        strata=strata, 
        compartments=infectious_comps
    )  
    
    # split between the different spredear categories (c := prop of superspreader)
    c = Parameter("c")
    split_props = {
        "subspreader": 1 - c,
        "superspreader": c
    }
    strat.set_flow_adjustments(
        "infection",
        split_props
    )

    rho = Parameter("rho")
    # adjust the infectiousness level of the different categories
    infectiousness_adjustments = {
        "subspreader": rho,
        "superspreader": 1.
    }        
    for infectious_comp in infectious_comps:
        strat.add_infectiousness_adjustments(
            infectious_comp,
            adjustments=infectiousness_adjustments
        )

    return strat



In [ ]:
def build_full_model():

    # Get an unstratified model object
    model = build_unstratified_model()
    
    base_compartments = model.compartments

    # Get and apply the infectiousness stratification
    infectiousness_strat = get_infectiousness_stratification()
    model.stratify_with(infectiousness_strat)
    
    return model

# Example run, also used to generate dummy data

In [ ]:
model = build_full_model()

In [ ]:
model.get_input_parameters()

In [ ]:
parameters = {    
    'beta': .2, 
    'infectious_period': 7., 
    'infectious_seed': 100.,
        
    # spreader categories
    'c': .75,
    'rho': .5
}


In [ ]:
model.run(parameters)
raw_incidence_target = model.get_derived_outputs_df()["incidence"] 
raw_incidence_target.plot()

### Add some noise to the data

In [ ]:
incidence_target = raw_incidence_target + np.random.normal(loc=0,scale=2000,size=len(raw_incidence_target))
incidence_target[incidence_target < 0.] = 0.
incidence_target.plot()

## Automatic calibration

In [ ]:
from estival import priors, targets
from estival.calibration.mcmc.adaptive import AdaptiveChain

In [ ]:
def run_calibration(iterations):
    # define calibration targets
    mcmc_targets = [
        targets.NormalTarget("incidence", incidence_target, priors.UniformPrior("target_sd", [1000, 10000])) 
    ]

    # define priors
    mcmc_priors = [
        priors.UniformPrior("beta", [0.1, 0.5]),
        priors.UniformPrior("c", [0., 1.]),
        priors.UniformPrior("rho", [0., 1.]),
    ]         

    # define initial parameters
    parameters = {
        'beta': .3, 
        'infectious_period': 7., 
        'infectious_seed': 100.,
        'c': .75,
        'rho': .5,
        'target_sd': 5000
    }
    init_p = parameters.copy()

    mcmc = AdaptiveChain(build_full_model, parameters, mcmc_priors, mcmc_targets, init_p, adaptive_proposal=False)
    mcmc.run(max_iter=iterations)

    return mcmc

In [ ]:
n_iterations = 20000
mcmc = run_calibration(n_iterations)

In [ ]:
mcmc.n_accepted

In [ ]:
import arviz as az

In [ ]:
# Some values to adjust to produce the desired outputs
burn_in_prop = 0.5
sample_for_plot = 50

In [ ]:
burn_in = round(burn_in_prop * n_iterations)  # Find the integer number of burn-in iterations

burnt_results = mcmc.results[burn_in:]  # Get the MCMC results after burn-in
accepted_mcmc = [burnt_results[i] for i in range(len(burnt_results)) if burnt_results[i].accept]  # Extract the accepted iterations


In [ ]:
mcmc_sample = random.sample(accepted_mcmc, sample_for_plot)  # Choose a sample to run for plotting later

inf_data = mcmc.to_arviz(burn_in)  # Get the post-burn in chain in arviz format

In [ ]:
out_df = {}
recovered_df = {}
for i, r in enumerate(mcmc_sample):
    cur_params = parameters.copy()
    cur_params.update(r.parameters)
    model.run(cur_params)
    derived_out = model.get_derived_outputs_df()
    out_df[i] = derived_out["incidence"]


In [ ]:
pd.options.plotting.backend = "matplotlib"
ax = pd.DataFrame(out_df).plot(style='-', figsize=(15, 6), legend=False)
incidence_target.plot(style='.', color="black")
pd.options.plotting.backend = "plotly"

In [ ]:
# Find the parameter set with the highest log likelihood obtained

best_ll = -np.inf
best_res = None

for r in mcmc.results:
    if r.ll > best_ll:
        best_ll = r.ll
        best_res = r
        
best_res.parameters

In [ ]:
max_ll_params = parameters.copy()
max_ll_params.update(best_res.parameters)

model.run(max_ll_params)

comparison_df = pd.DataFrame({
    "modelled": model.get_derived_outputs_df()["incidence"],
    "reported": incidence_target,
})
comparison_df.plot()

In [ ]:
az.summary(inf_data)

In [ ]:
az.plot_trace(inf_data, figsize=(16, 19));

In [ ]:
az.plot_posterior(inf_data);

In [ ]:
az.plot_pair(inf_data)

In [ ]:
from estival.utils import to_df, to_arviz

In [ ]:
caldf = to_df(mcmc)

In [ ]:
px.line(caldf, x="iteration", y="log_likelihood")